In [1]:
import sys
sys.path.append('/home/iiyama/src/fastdla')
from collections.abc import Sequence
import numpy as np
from fastdla.dla import generate_dla, is_independent
from fastdla.sparse_pauli_vector import SparsePauliVector

In [2]:
def z2lgt_hva_generators(num_fermions: int) -> list[SparsePauliVector]:
    """Construct generators of the HVA for the Z2 LGT model for the given number of staggered
    fermions."""
    num_qubits = 4 * num_fermions

    generators = []

    # Mass terms Z_{n}
    for parity in [0, 1]:
        strings = ['I' * (num_qubits - isite * 2 - 1) + 'Z' + 'I' * (isite * 2)
                   for isite in range(parity, 2 * num_fermions, 2)]
        coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
        generators.append(SparsePauliVector(strings, coeffs))

    # Field term X_{n,n+1}
    strings = ['I' * (num_qubits - iq - 1) + 'X' + 'I' * iq for iq in range(1, num_qubits, 2)]
    coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
    generators.append(SparsePauliVector(strings, coeffs))

    # Hopping terms X_{n}Z_{n,n+1}X_{n+1} + Y_{n}Z_{n,n+1}Y_{n+1}
    for parity in [0, 1]:
        strings = []
        for isite in range(parity, 2 * num_fermions, 2):
            for site_op in ['X', 'Y']:
                paulis = ['I'] * num_qubits
                paulis[isite * 2] = site_op
                paulis[isite * 2 + 1] = 'Z'
                paulis[(isite * 2 + 2) % num_qubits] = site_op
                strings.append(''.join(paulis[::-1]))
        coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
        generators.append(SparsePauliVector(strings, coeffs))

    return generators


In [3]:
def z2lgt_gauss_projector(charges: Sequence[int]) -> SparsePauliVector:
    """Construct the Gauss's law projectors for the Z2 LGT model.

    Physical states of the Z2 LGT model must be eigenstates of G_n = X_{n-1,n}Z_{n}X_{n1,n+1}. Given
    an eigenvalue for each G_n, we can construct projector to its subspace as a sum of Pauli ops.
    The overall projector will then be a product of all such projectors.
    """
    if len(charges) % 2 or not all(abs(ev) == 1 for ev in charges):
        raise ValueError('There must be an even number of charges with values +-1')

    num_fermions = len(charges) // 2
    num_qubits = 4 * num_fermions

    projector = SparsePauliVector('I' * num_qubits, 1.)
    for isite, ev in enumerate(charges):
        if ev > 0:
            # +0+ -1+ -0- +1- with +/-=1/2(I±X) and 0/1=1/2(I±Z)
            # From
            # np.sum([
            #     np.kron(np.kron([1, 1], [1, 1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, -1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, 1]), [1, -1]),
            #     np.kron(np.kron([1, 1], [1, -1]), [1, -1])
            # ]) / 8
            # we get array([0.5, 0, 0, 0, 0, 0, 0, 0.5]) -> 1/2(III + XZX)

            coeffs = [0.5, 0.5]
        else:
            # -1- +0- +1+ -0+ with +/-=1/2(I±X) and 0/1=1/2(I±Z)
            # From
            # np.sum([
            #     np.kron(np.kron([1, 1], [1, 1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, -1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, 1]), [1, -1]),
            #     np.kron(np.kron([1, 1], [1, -1]), [1, -1])
            # ]) / 8
            # we get array([0.5, 0, 0, 0, 0, 0, 0, -0.5]) -> 1/2(III - XZX)
            coeffs = [0.5, -0.5]
        strings = ['I' * num_qubits]
        paulis = ['I'] * num_qubits
        paulis[-isite * 2] = 'X'
        paulis[-isite * 2 - 1] = 'Z'
        paulis[-((isite * 2 + 2) % num_qubits)] = 'X'
        strings.append(''.join(paulis))
        projector = projector @ SparsePauliVector(strings, coeffs)

    return projector

In [4]:
def z2lgt_u1_projector(num_fermions: int, charge: int) -> SparsePauliVector:
    """Construct the charge conservation law projector for the Z2 LGT model.

    Physical states of the Z2 LGT model must be eigenstates of Q = ∑_n Z_n where n runs over
    staggered fermions (even n: particle, odd n: antiparticle). For a fermion number F and an
    overall charge q ∈ [-2F,...,2F], we have a 2F C (F+q/2) -dimensional eigenspace.
    """
    if abs(charge) > (num_sites := 2 * num_fermions) or charge % 2 != 0:
        raise ValueError('Invalid charge value')

    # Diagonals=eigenvalues of the symmetry generator
    eigvals = np.zeros((2,) * num_sites, dtype=int)
    z = np.array([1, -1])
    for isite in range(num_sites):
        eigvals += np.expand_dims(z, tuple(range(isite)) + tuple(range(isite + 1, num_sites)))
    eigvals = eigvals.reshape(-1)
    # State indices with the given charge
    states = np.nonzero(eigvals == charge)[0]
    # Binary representations of the indices
    states_binary = (states[:, None] >> np.arange(num_sites)[None, ::-1]) % 2
    # |0><0|=1/2(I+Z), |1><1|=1/2(I-Z) -> Coefficients of I and Z for each binary digit
    # Example: [0, 1] -> [[1, 1], [1, -1]]
    states_iz = np.array([[1, 1], [1, -1]])[states_binary]
    # Take the kronecker products of the I/Z coefficients using einsum, then sum over the states to
    # arrive at the final sparse Pauli representation of the projector
    args = ()
    for isite in range(num_sites):
        args += (states_iz[:, isite], [0, isite + 1])
    args += (list(range(num_sites + 1)),)
    coeffs = np.sum(np.einsum(*args).reshape(states.shape[0], 2 ** num_sites), axis=0)
    # Take only the nonzero Paulis
    pauli_indices = np.nonzero(coeffs)[0]
    coeffs = coeffs[pauli_indices] / (2 ** num_sites)
    paulis = []
    for idx in pauli_indices:
        idx_bin = np.zeros((num_sites, 2), dtype=int)
        idx_bin[:, 1] = (idx >> np.arange(num_sites)[::-1]) % 2
        paulis.append(''.join('IZ'[i] for i in idx_bin.reshape(-1)))

    return SparsePauliVector(paulis, coeffs)

In [5]:
# Determine the charge sector (symmetry subspace) to investigate
charges = [1, -1, 1, -1]
num_fermions = len(charges) // 2
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)


In [6]:
generators_full

[SparsePauliVector(['IIIIIIIZ', 'IIIZIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIZII', 'IZIIIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIIXI', 'IIIIXIII', 'IIXIIIII', 'XIIIIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIIIXZX', 'IIIIIYZY', 'IXZXIIII', 'IYZYIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIXZXII', 'IIIYZYII', 'ZXIIIIIX', 'ZYIIIIIY'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j]))]

In [7]:
# Compute the DLA of the full space
dla_full = generate_dla(generators_full, verbosity=1)
print(f'DLA dimension is {len(dla_full)}')

Current DLA dimension: 10
Current DLA dimension: 23
Current DLA dimension: 103
Current DLA dimension: 209
Current DLA dimension: 311
DLA dimension is 311


In [8]:
# Projectors
gauss_projector = z2lgt_gauss_projector(charges)
u1_projector = z2lgt_u1_projector(num_fermions, sum(charges))
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [9]:
# Check the dimensionality of the symmetry subspace
proj_eigvals = np.linalg.eigh(symm_projector.to_matrix()).eigenvalues
print(f'Hilbert subspace dimension is {np.nonzero(np.isclose(proj_eigvals, 1.))[0].shape[0]}')

Hilbert subspace dimension is 12


In [10]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [11]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=1)
print(f'Subspace DLA dimension is {len(dla)}')

Current DLA dimension: 5
Current DLA dimension: 8
Current DLA dimension: 15
Current DLA dimension: 33
Current DLA dimension: 49
Current DLA dimension: 52
Current DLA dimension: 53
Current DLA dimension: 55
Current DLA dimension: 60
Current DLA dimension: 63
Current DLA dimension: 64
Subspace DLA dimension is 64
